In [2]:
!pip install pmdarima

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller, acf

from scipy.fft import fft
from scipy import signal

from sklearn.datasets import make_classification
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score, classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import accuracy_score
from sklearn import metrics


from pmdarima.model_selection import train_test_split as time_train_test_split
from pmdarima.arima import auto_arima
import pmdarima as pm

from imblearn.over_sampling import RandomOverSampler

from prophet import Prophet

from xgboost import XGBRegressor

from google.colab import drive

import pandas as pd
import numpy as np

import os

import joblib

import statsmodels.api as sm

import warnings
warnings.filterwarnings("ignore")

**UTILITY FUNCTION**

In [4]:
def mean_absolute_percentage_error_func(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [5]:
def mount_drive():
  drive.mount('/content/drive')

In [6]:
mount_drive()

Mounted at /content/drive


**EXTRACT FEATURES FROM THE TIME SERIES DATASET**

In [7]:
def get_features(df):
  # Set date column as the index
  df.set_index('ds', inplace=True)

  # Extract time series features
  features = pd.DataFrame()
  features['mean'] = df.mean()
  features['std'] = df.std()
  features['min'] = df.min()
  features['max'] = df.max()
  features['median'] = df.median()
  features['kurtosis'] = df.kurtosis()
  features['skewness'] = df.skew()
  features['quantile_25'] = df.quantile(q=0.25)
  features['quantile_75'] = df.quantile(q=0.75)
  features['range'] = df.max() - df.min()
  features['interquartile_range'] = features['quantile_75'] - features['quantile_25']
  features['variation_coefficient'] = features['std'] / features['mean']
  
  # Compute the autocorrelation score
  '''acf_vals  = acf(df['y'], fft=True)
  autocorr_score = max(acf_vals)
  features['autocorr_score'] = autocorr_score
  '''
  lags = range(31)
  autocorr = acf(df, nlags=30)
  autocorr_score=np.abs(np.mean(autocorr))
  features['autocorr_score'] = autocorr_score

  # Identify and count outliers using the interquartile range (IQR) method
  #q1 = df.quantile(0.25)
  q1 = df['y'].quantile(0.25)
  #q3 = df.quantile(0.75)
  q3 = df['y'].quantile(0.75)
  iqr = q3 - q1
  outliers = ((df < (q1 - 1.5 * iqr)) | (df > (q3 + 1.5 * iqr))).sum(axis=1)
  features['outliers'] = outliers

  # Compute the stationarity score
  adf_test = adfuller(df['y'])
  stationarity_score = adf_test[1]
  features['stationarity_score'] = stationarity_score

  # Compute the spectral density score using Welch's method
  freqs, psd = signal.welch(df['y'], nperseg=256)
  spectral_density_score = np.sum(psd)
  features['spectral_density_score'] = spectral_density_score

  # Compute Noise score
  window_size = 30
  detrended = signal.detrend(df['y'], type='linear')
  noise_variance = np.var(detrended)
  residuals = df['y'] - detrended
  noise_score = np.std(residuals)
  features['noise_score'] = noise_score

  # Extract seasonality, trend and residual components using seasonal_decompose
  try:
    result = seasonal_decompose(df, period = 12)
    features['trend'] = result.trend.mean()
    features['residual'] = result.resid.mean()
    features['seasonality'] = result.seasonal.mean()
  except:
    features['seasonality'] = np.nan
  
  # Print extracted features
  return features

**PROPHET MODEL**

In [8]:
def fit_prophet(df):
  df['ds'].dt.tz_localize(None)

  # Split data into training and testing sets
  train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

  # Create a Prophet model instance and fit it to the training data
  model = Prophet()
  model.fit(train_data)

  # Make predictions on the test set
  future = model.make_future_dataframe(periods=len(test_data))
  forecast = model.predict(future)
  forecast = forecast[-len(test_data):] # Only keep the forecasted values for the test set

  # Calculate MAPE
  
  mape = mean_absolute_percentage_error_func(np.array(test_data['y']), np.array(forecast['yhat']))

  return mape

**ETS MODEL**

In [9]:
def fit_ets(df):
  df.set_index('ds', inplace=True)


  # Split data into train and test sets
  train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

  # fit the ETS model on the training data
  model = sm.tsa.ExponentialSmoothing(train_data['y'], trend='add', seasonal='add', seasonal_periods=12).fit()

  # make predictions on the test data
  predictions = model.forecast(len(test_data))

  mape = mean_absolute_percentage_error_func(np.array(test_data['y']), np.array(predictions))
  
  return mape

**ARIMA MODEL**

In [10]:
def fit_ARIMA(df):
  df.set_index('ds', inplace=True)
  y = df['y'].values

  train, test = train_test_split(y, test_size=0.2, shuffle=False)

  arima_model = pm.auto_arima(train,    
      seasonal=False,
      suppress_warnings=True,
      error_action='ignore',
      start_p=0, d=None, start_q=0,
      max_p=5, max_d=2, max_q=5,
      test="adf"
  ) 
  
  arima_model.fit(train)  
  y_pred = arima_model.predict(n_periods=len(test))
  
  mape = mean_absolute_percentage_error_func(np.array(test), np.array(y_pred))

  return mape

**XGBoost MODEL**

In [11]:
def create_features(df, target_variable):
    df['date'] = df.index
    df['hour'] = df['date'].dt.hour
    df['dayofweek'] = df['date'].dt.dayofweek
    df['quarter'] = df['date'].dt.quarter
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year
    df['dayofyear'] = df['date'].dt.dayofyear
    df['dayofmonth'] = df['date'].dt.day
    df['weekofyear'] = df['date'].dt.weekofyear
    
    X = df[['hour','dayofweek','quarter','month','year',
           'dayofyear','dayofmonth','weekofyear']]
    if target_variable:
        y = df[target_variable]
        return X, y
    return X

In [12]:
def fit_XGB(df):
  # Make sure that you have the correct order of the times 
  df = df.sort_values(by='ds', ascending=True)

  # Set Datetime as index
  df = df.set_index('ds')

  X = df['y']

  # Test Size = 20%
  train_df, test_df = time_train_test_split(X, test_size=int(len(df)*0.2))

  train_df = pd.DataFrame(train_df)
  test_df = pd.DataFrame(test_df)

  train_df_copy = train_df.copy()
  test_df_copy = test_df.copy()

  trainX, trainY = create_features(train_df_copy, target_variable='y')
  testX, testY = create_features(test_df_copy, target_variable='y')

  xgb = XGBRegressor(objective= 'reg:linear', n_estimators=1000)

  xgb.fit(trainX, trainY,
          eval_set=[(trainX, trainY), (testX, testY)],
          early_stopping_rounds=50,
          verbose=False)
  
  predicted_results = xgb.predict(testX)
  
  return mean_absolute_percentage_error_func(testY, predicted_results)


**GET THE LABEL**

In [13]:
def get_label(df):
  mape = []
  
  mape.append(fit_prophet(df.copy()))
  mape.append(fit_ets(df.copy()))
  mape.append(fit_XGB(df.copy()))
  mape.append(fit_ARIMA(df.copy()))

  print(mape)

  return mape.index(min(mape))

In [ ]:
features_df = pd.DataFrame()
labels_df = pd.Series()

folder_path = '/content/drive/MyDrive/Time Series Data/'

# Iterate through all files in the folder
for file_name in os.listdir(folder_path):
    # Check if file is a CSV file
    if file_name.endswith('.csv'):
        # Do something with the file
        file_path = os.path.join(folder_path, file_name)
        
        print(file_path)

        df = pd.read_csv(file_path)

        if 'Unnamed: 0' in df.columns:
          df = df.drop(columns = 'Unnamed: 0', axis = 1)


        df.columns.values[0] = 'ds'
        df.columns.values[1] = 'y'

        print(df.head())

        #Impute point_value
        df['y'] = df['y'].fillna(df['y'].mean())
        
        features_df = pd.concat([features_df, get_features(df.copy())], ignore_index=True)

        df['ds'] = pd.to_datetime(df['ds'])
        df['ds'] = df['ds'].dt.tz_localize(None)

        
        t = get_label(df.copy())
        print("----", t)

        labels_df = labels_df.append(pd.Series([t]))

In [ ]:
features_df.to_csv('features.csv')

In [ ]:
labels_df.to_csv('labels.csv')

In [ ]:
features_df.head()

,mean,std,min,max,median,kurtosis,skewness,quantile_25,quantile_75,range,interquartile_range,variation_coefficient,autocorr_score,outliers,stationarity_score,spectral_density_score,noise_score,trend,residual,seasonality
0,0.080813,0.007967,0.058061,1.111919e-01,0.080898,0.104336,0.171542,0.075316,0.085777,5.313133e-02,0.010461,0.098582,0.167094,NaN,5.674967e-11,1.594984e-02,0.000142,0.080858,0.000005,2.606377e-06
1,49.848643,3.085998,43.146586,5.917162e+01,49.848643,2.137306,0.737816,48.329033,51.009674,1.602504e+01,2.680641,0.061907,0.010795,NaN,9.476016e-04,4.483121e+02,0.011606,49.866497,0.069493,1.086313e-01
2,78.977742,2.048750,73.599728,8.281616e+01,79.291599,0.313815,-0.785742,78.048408,80.359825,9.216432e+00,2.311417,0.025941,0.331286,NaN,3.052435e-01,5.705726e+02,0.865636,78.968035,0.024176,2.548743e-03
3,456027.723629,216471.561459,1.000000,1.177448e+06,522566.000000,-0.789263,-0.460697,245346.000000,610860.000000,1.177447e+06,365514.000000,0.474689,0.033316,NaN,1.327504e-06,1.219912e+13,6614.732967,457235.694720,535.452180,2.640457e+00
4,568.048860,57.713088,307.389626,1.531098e+03,568.048860,100.056566,6.395766,543.327542,585.242382,1.223709e+03,41.914840,0.101599,0.127311,NaN,2.505346e-03,6.830140e+05,16.104514,568.189593,0.045114,1.879743e-16


**TRAINING CLASSIFICATION MODEL**

In [22]:
result = pd.read_csv('/content/drive/MyDrive/TSA FEATURE DATASET/final_dataset.csv')
result = result.drop(columns = ['Unnamed: 0'], axis = 1)

In [15]:
result.head()

,mean,std,min,max,median,kurtosis,skewness,quantile_25,quantile_75,range,...,variation_coefficient,autocorr_score,outliers,stationarity_score,spectral_density_score,noise_score,trend,residual,seasonality,label
0,0.080813,0.007967,0.058061,1.111919e-01,0.080898,0.104336,0.171542,0.075316,0.085777,5.313133e-02,...,0.098582,0.167094,NaN,5.670000e-11,1.594984e-02,0.000142,0.080858,0.000005,2.610000e-06,1
1,49.848643,3.085998,43.146586,5.917162e+01,49.848643,2.137306,0.737816,48.329033,51.009674,1.602503e+01,...,0.061907,0.010795,NaN,9.476016e-04,4.483121e+02,0.011606,49.866497,0.069493,1.086313e-01,2
2,78.977742,2.048750,73.599728,8.281616e+01,79.291599,0.313815,-0.785742,78.048408,80.359825,9.216432e+00,...,0.025941,0.331286,NaN,3.052435e-01,5.705726e+02,0.865636,78.968035,0.024176,2.548743e-03,2
3,456027.723600,216471.561500,1.000000,1.177448e+06,522566.000000,-0.789263,-0.460697,245346.000000,610860.000000,1.177447e+06,...,0.474689,0.033316,NaN,1.330000e-06,1.219912e+13,6614.732967,457235.694700,535.452180,2.640457e+00,1
4,568.048861,57.713088,307.389626,1.531098e+03,568.048861,100.056566,6.395766,543.327542,585.242382,1.223709e+03,...,0.101599,0.127311,NaN,2.505346e-03,6.830140e+05,16.104514,568.189593,0.045114,1.880000e-16,1


In [23]:
result = result.drop(columns = ['mean', 'std', 'min', 'max', 'median', 'kurtosis', 'skewness', 'quantile_25', 'quantile_75', 'range', 'interquartile_range', 'variation_coefficient'])

In [17]:
# get the count of each label
result['label'].value_counts()

2    26
1    11
3    10
0     7
Name: label, dtype: int64

In [24]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler

def clean_data(X):
  # remove features that have all NaN values
  X = X.dropna(axis='columns', how='all')

  # create a pipeline to impute missing values with the mean and apply standard scaling
  num_pipeline = Pipeline([
      ('imputer', SimpleImputer(strategy='mean')),
      ('scaler', MinMaxScaler())
  ])

  # fit_transform the pipeline on the dataset
  X_transformed = num_pipeline.fit_transform(X)

  return X_transformed

In [ ]:
X_transformed = clean_data(result.drop(columns = ['label'], axis = 1))

y = result['label']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size=0.2, random_state=42)

# Create an instance of the RandomOverSampler class
ros = RandomOverSampler()

# Resample the training data
X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)

# Define classification models
models = [
    ('Random Forest', RandomForestClassifier()),
    ('AdaBoost', AdaBoostClassifier()),
    ('Gradient Boosting', GradientBoostingClassifier()),
    ('Logistic Regression', LogisticRegression()),
    ('Gaussian Naive Bayes', GaussianNB()),
    ('Decision Tree', DecisionTreeClassifier()),
    ('K-Nearest Neighbors', KNeighborsClassifier()),
    ('Multi-layer Perceptron', MLPClassifier()),
    ('Support Vector Machine', SVC())
]

# Define hyperparameters for each model
params = [
    {'n_estimators': [100, 500, 1000], 'max_depth': [5, 10, 20], 'min_samples_leaf': [1, 5, 10]},
    {'n_estimators': [100, 500, 1000], 'learning_rate': [0.1, 0.5, 1]},
    {'n_estimators': [100, 500, 1000], 'learning_rate': [0.1, 0.5, 1], 'max_depth': [3, 5, 10]},
    {'C': [0.1, 1, 10], 'penalty': ['l1', 'l2']},
    {},
    {'max_depth': [5, 10, 20], 'min_samples_leaf': [1, 5, 10]},
    {'n_neighbors': [3, 5, 7], 'weights': ['uniform', 'distance']},
    {'hidden_layer_sizes': [(100,), (100, 100), (100, 100, 100)], 'activation': ['relu', 'tanh']},
    {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
]

# Fit models and apply hyperparameter tuning
best_model = None
best_score = 0

i = 0
for name, model in models:
    grid = GridSearchCV(model, param_grid=params[i], scoring='accuracy', cv=5, n_jobs=-1)
    grid.fit(X_train_resampled, y_train_resampled)
    score = grid.best_score_
    if score > best_score:
        best_score = score
        best_model = grid.best_estimator_
    print(f"{grid.best_estimator_} Accuracy: {accuracy_score(y_test, grid.predict(X_test))}")
    i += 1
    
print(f"\nBest Model: {best_model}")


RandomForestClassifier(max_depth=20) Accuracy: 0.45454545454545453
AdaBoostClassifier(learning_rate=0.5, n_estimators=500) Accuracy: 0.5454545454545454
GradientBoostingClassifier(learning_rate=1, n_estimators=1000) Accuracy: 0.36363636363636365
LogisticRegression(C=10) Accuracy: 0.36363636363636365
GaussianNB() Accuracy: 0.09090909090909091
DecisionTreeClassifier(max_depth=20) Accuracy: 0.5454545454545454
KNeighborsClassifier(n_neighbors=3, weights='distance') Accuracy: 0.2727272727272727
MLPClassifier(hidden_layer_sizes=(100, 100)) Accuracy: 0.36363636363636365
SVC(C=10) Accuracy: 0.2727272727272727

Best Model: RandomForestClassifier(max_depth=20)


In [31]:
X_transformed = clean_data(result.drop(columns = ['label'], axis = 1))

y = result['label']

# split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size=0.3, random_state=42)

# Create an instance of the RandomOverSampler class
ros = RandomOverSampler()

# Resample the training data
X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)

# define a list of classifiers to evaluate
classifiers = [
    LogisticRegression(),
    MultinomialNB(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    GradientBoostingClassifier(),
    KNeighborsClassifier(),
    MLPClassifier(),
    SVC()
]


# define an empty dictionary to store the accuracy scores of each classifier
best_score = 0
best_model = None

# fit each classifier and store its accuracy score
for clf in classifiers:
    # fit the pipeline on the training data
    clf.fit(X_train_resampled, y_train_resampled)
    
    # make predictions on the test data
    y_pred = clf.predict(X_test)
    
    # calculate the accuracy score and store it in the dictionary
    model_accuracy_score = accuracy_score(y_test, y_pred)

    if(best_score <= model_accuracy_score):
      best_score = model_accuracy_score
      best_model = clf

    print(str(clf), model_accuracy_score)

# print the best classifier
print(f"\nThe best classifier is {best_model} with an accuracy score of {best_score}")

LogisticRegression() 0.29411764705882354
MultinomialNB() 0.35294117647058826
DecisionTreeClassifier() 0.5294117647058824
RandomForestClassifier() 0.5882352941176471
GradientBoostingClassifier() 0.5294117647058824
KNeighborsClassifier() 0.47058823529411764
MLPClassifier() 0.4117647058823529
SVC() 0.4117647058823529

The best classifier is RandomForestClassifier() with an accuracy score of 0.5882352941176471


In [32]:
model_list = [
    fit_prophet,
    fit_ets,
    fit_XGB,
    fit_ARIMA
]

df = pd.read_csv("/content/drive/MyDrive/Time Series Data/sample (26).csv")

if 'Unnamed: 0' in df.columns:
          df = df.drop(columns = 'Unnamed: 0', axis = 1)


df.columns.values[0] = 'ds'
df.columns.values[1] = 'y'

#Impute point_value
df['y'] = df['y'].fillna(df['y'].mean())

feature_df = get_features(df.copy())
feature_df = feature_df.drop(columns = ['mean', 'std', 'min', 'max', 'median', 'kurtosis', 'skewness', 'quantile_25', 'quantile_75', 'range', 'interquartile_range', 'variation_coefficient'])
print(feature_df)

feature_df = clean_data(feature_df)

df['ds'] = pd.to_datetime(df['ds'])
df['ds'] = df['ds'].dt.tz_localize(None)

print("0", fit_prophet(df.copy()))
print("1", fit_ets(df.copy()))
print("2", fit_XGB(df.copy()))
print("3", fit_ARIMA(df.copy()))

print("RESULT ", best_model.predict(feature_df)[0])

#model_list[best_model.predict(feature_df)[0]](df.copy())

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


   autocorr_score  outliers  stationarity_score  spectral_density_score  \
y        0.034909       NaN                 0.0            9.439851e+06   

   noise_score        trend  residual  seasonality  
y     2.490481  1011.057574 -0.005763     0.009562  


DEBUG:cmdstanpy:input tempfile: /tmp/tmp8x0yffbm/tg2omnfr.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp8x0yffbm/oa37cziz.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.9/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=53039', 'data', 'file=/tmp/tmp8x0yffbm/tg2omnfr.json', 'init=/tmp/tmp8x0yffbm/oa37cziz.json', 'output', 'file=/tmp/tmp8x0yffbm/prophet_modelwz2kv_10/prophet_model-20230312162634.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
16:26:34 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
16:26:35 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


0 5.149606320095143
1 1.0197220243936045
[16:26:37] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
2 143.43262791062503
3 144.68202584836965
RESULT  0


In [33]:
# Save the model to a file using joblib
joblib.dump(best_model, '/content/drive/MyDrive/MODELS/BestModel.pkl')

['/content/drive/MyDrive/MODELS/BestModel.pkl']

In [34]:
# Load the model from a file using joblib
rf = joblib.load('/content/drive/MyDrive/MODELS/BestModel.pkl')

# Use the loaded model to make predictions
y_pred = rf.predict(feature_df)

In [35]:
y_pred

array([0])